<a href="https://colab.research.google.com/github/ViniciusSiqueira54/InteractiveTestBook/blob/master/Data_Analysis/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Authenticate block

In [ ]:
'''
Auth block of code

In this block provide your credentials to the runtime, notice that to use this 
you need to use a Motorola account to have access to the BigQuery data used here.

For further purposes, this block needs to be changed to authenticate using the 
Google Cloud API for testing purposes or authenticate as an end-user, 
for more information on how to do this check the links below.

https://cloud.google.com/docs/authentication/getting-started
https://cloud.google.com/docs/authentication/end-user
'''

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

#Import Libs

In [ ]:
!pip install --upgrade gspread
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import requests
import gspread
import time

from datetime import datetime

class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
# Importing the dtExtractFromBQ.py from github, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/mirandaRG/hbprio/master/Data_Analysis/Data_Extraction/dtExtractFromBQ.py?token=AO47UI2O4LSCLX4OGXVB743BTLLN2'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('dtExtractFromBQ.py', 'w') as f:
    f.write(r.text)

import dtExtractFromBQ as bq

In [ ]:

# Importing the metricsHBPrio.py from github, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/mirandaRG/hbprio/master/Data_Analysis/Data_Extraction/metricsHBPrio.py?token=AO47UIYK2QTA5FGOHVADXFDBTLLO6'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('metricsHBPrio.py', 'w') as f:
    f.write(r.text)

# now we can import
import metricsHBPrio as mt

#APFD Analysis (Core Plans)

## One Plan

In [ ]:
def APFDResults(): 
  print("Check below the comparison of different APFD heuristics.\n")

  ro_med = 0

  for x in range(0, 30):
      ro_med += mt.RandomlyOrdered()

  ro_med = ro_med/30

  print("\n\tAPFD value of the " +color.BOLD+ "Original Test Date" +color.END+ " used by the testers: %.2f" %(mt.OrderedTestDate()))
  print("\tAPFD value if testers had followed the " +color.BOLD+ "Default Dalek Ordenation" +color.END+ ": %.2f" %(mt.OrderedDalekDefault()))
  print("\tAPFD value of some " +color.BOLD+ "Randomly Ordination" +color.END+ " of the test results: %.2f" %(ro_med))
  print("\tAPFD value of the " +color.BOLD+ "Best Possible Ordination" +color.END+ " (Fail and Blocked by CRs first): %.2f" %(mt.BestOrdered()))
  print("\n\tAPFD value of the " +color.BOLD+ "Case Creation Order from Older to Newer cases" +color.END+ ": %.2f" %(mt.OlderToNew()))
  print("\tAPFD value of the " +color.BOLD+ "Case Creation Order from Newer to Older cases" +color.END+ ": %.2f" %(mt.NewToOlder()))
  print("\n\n\tAPFD value of the " +color.BOLD+ "HBprio Trial" +color.END+ " (Cases with higher failure percentage first): %.2f" %(mt.OrderedHistoryBase()))

# -- APFDResults

In [ ]:
mt.preAPFD(planDF)
APFDResults()

## Multiple Plans (Core Plans)

In [ ]:
def APFDResults2(append_index, plan): 
 
  ro_med = 0

  for x in range(0, 30):
      ro_med += mt.RandomlyOrdered()

  ro_med = round(ro_med/30, 2)
  otd = round(mt.OrderedTestDate(), 2)
  odd = round(mt.OrderedDalekDefault(), 2)
  bo = round(mt.BestOrdered(), 2)
  otn = round(mt.OlderToNew(), 2)
  nto = round(mt.NewToOlder(), 2)
  ohb = round(mt.OrderedHistoryBase(), 2)

  ind = append_index;
  df = pd.DataFrame({"Device": ["Test"],"TP Id": [plan],"OTD": [otd],"ODD": [odd],"RO": [ro_med],"BO": [bo],"OtN": [otn],"NtO": [nto],"OHB": [ohb]},index=[ind])

  return df
  
# -- APFDResults2

In [ ]:
#plans = ["MCA-1872700","MCA-2738679","MCA-2454353","MCA-2454354","MCA-2752240","MCA-2454250","MCA-2708462","MCA-2742232","MCA-2808712","MCA-2708394"]
#plans = ["MCA-2138838","MCA-2137314","MCA-2138839","MCA-1930879","MCA-2271108"]


global dataframe_collection, testplanid_collection, product
dataframe_collection = []
testplanid_collection = [] 

CoreTrial_df = pd.DataFrame()
x=0

for i in plans:
  updateQueries(i)
  mt.preAPFD(getPlanData())
  df = APFDResults2(x,i)
  x+=1
  CoreTrial_df = CoreTrial_df.append(df)

#result = df1.append(df2)

###Save output to HBPrio Trial Spreadsheet 

[HBPrio Trial Spreadsheet](https://docs.google.com/spreadsheets/d/11lHXTsAPC_40kiSXIjyHPl7RHRT6N2NzOjA41HdlsYM/edit#gid=1426537970)

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

#copia todo o workbook pra essa variavel 
wbfull = gc.open_by_key("11lHXTsAPC_40kiSXIjyHPl7RHRT6N2NzOjA41HdlsYM")

today = datetime.now().strftime('%d/%m/%Y|%H:%M')
newSheetName = 'Trial Execution Record - '+ today+'(utc)'

wbfull.duplicate_sheet(1426537970,new_sheet_name=newSheetName)

#PASSANDO OS DADOS PRA NOVA ABA
newSheet = wbfull.worksheet(newSheetName)
plansInfosList = CoreTrial_df.values.tolist()
newSheet.update('C3:L27',plansInfosList)

###Plota hotmap do exemplo acima

In [ ]:
import matplotlib.pyplot as plt
from pandas import DataFrame
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(12,10))
sns.heatmap(CoreTrial_df[['OTD', 'ODD', 'RO', 'BO', 'OtN', 'NtO', 'OHB', ]], cmap="Blues", annot=True, linecolor='white', linewidths=0.5)

## Multiple Plans (Core Plans by Family Product)

In [ ]:
def APFDResults3(append_index, plan, date): 

  print("Data dentro do APFDResults3")
  print(date)

#---------------------------------------------------------------------------------tempo#
  init_time = time.time()
#--------------------------------------------------------------------------------------#

  ro_med = 0
  mesmo, diferente = mt.CasesIdentifier(reorderedPlans,append_index,date) #used on ohb_02

  for x in range(0, 30):
      ro_med += mt.RandomlyOrdered()

  ro_med = round(ro_med/30, 2)
  otd = round(mt.OrderedTestDate(), 2)
  bo = round(mt.BestOrdered(), 2)
  nto = round(mt.NewToOlder(), 2)
  ohb_01 = round(mt.OrderedHistoryBase01(), 2)
  ohb_02 = round(mt.OrderedHistoryBase02(mesmo, diferente), 2)

#---------------------------------------------------------------------------------tempo#
  end_time = time.time()
  prep_time = round(end_time - init_time, 2) #tempo
#--------------------------------------------------------------------------------------#

  df1 = pd.DataFrame({"Device": [mt.apfd_filtered['Product'][0]],"TP Id": [plan],"# TCs": [mt.numCases],"# Failures":[mt.m],"Unique Cases": [mt.numUniqueCases],"Same Cases":[mt.numSameCases],"Diff Cases":[mt.numDiffeCases],"Prep Time":[prep_time],"OTD": [otd],"RO": [ro_med],"BO": [bo],"NtO": [nto],"OHB_v01": [ohb_01],"Prio Time ohb01" : [mt.ohb1_prio_time],"OHB_v02": [ohb_02],"Prio Time ohb02" : [mt.ohb2_prio_time],"Initial Date": [date]},index=[append_index])
 


#--------------------------------------------------------------------------------------#

  #Product	Id	Approach	APFD	Prio Time
  global df2

  df2 = pd.DataFrame({
                      'Product': [mt.apfd_filtered['Product'][0],'---','---','---','---','---'],
                      'TP Id': [plan,'---','---','---','---','---'],
                      'Approach': ['OTD','RO','BO','NewOld','OHB_v01','OHB_v02'],
                      'APFD': [otd, ro_med, bo, nto, ohb_01, ohb_02],
                      'Prio Time': ['---','---','---','---',mt.ohb1_prio_time,mt.ohb2_prio_time]
                      },
                  index=['1','2','3','4','5','6'])

#--------------------------------------------------------------------------------------#

  return df1
  
# -- APFDResults3

In [ ]:
#plans = ["MCA-490610","MCA-456825","MCA-613615","MCA-1018360","MCA-1012809","MCA-1080835","MCA-1123868","MCA-1210708","MCA-1179367","MCA-1192800","MCA-1218873","MCA-1190585","MCA-1254805","MCA-1254829","MCA-1254782","MCA-1256854","MCA-1359793","MCA-1359794","MCA-1380127","MCA-1386292","MCA-1467220","MCA-1464744","MCA-1560065","MCA-1609087","MCA-1654651","MCA-1657032","MCA-1652420","MCA-1652416","MCA-1652418","MCA-1930879","MCA-2138838","MCA-2138839","MCA-2137314","MCA-2271108","MCA-2454354","MCA-2438034","MCA-2454250","MCA-2455967","MCA-2454569","MCA-2540411","MCA-2525220","MCA-2540412","MCA-2540459","MCA-2454353","MCA-2738679","MCA-2710025","MCA-2708394","MCA-2708465","MCA-2742232","MCA-2752240","MCA-2808712"]
plans = ["MCA-2455967","MCA-2454569","MCA-2540411"]
#plans = ["MCA-2455967"]

#REORDENA OS PLANOS DE ACORDO COM A DATA DE INICIO

PlansDateInfo = pd.DataFrame()
a=0
x=0

#used on CasesIdentifier()
global dataframe_collection, testplanid_collection, product
dataframe_collection = []
testplanid_collection = [] 

for i in plans:
  bq.getPlanInitDateQuery(i)
  df = bq.getPlanInitDate()
  df['PlanID'] = i
  df.rename(index={0:a},inplace=True)
  PlansDateInfo = PlansDateInfo.append(df)
  a+=1

PlansDateInfo['dataInicial'] = pd.to_datetime(PlansDateInfo['DateInit'])
PlansDateInfo = PlansDateInfo.sort_values(by='dataInicial', ascending=True)

reorderedPlans = PlansDateInfo['PlanID'].tolist()


In [ ]:
#print(reorderedPlans)

In [ ]:
#PlansDateInfo.head(30)

In [ ]:
# ORIGINAL CELL

#%%time

Moto_G_Family = ['Rio H - Lake','Oslo M','Rio M - River','Oslo H - Ocean','Doha - Doha','RAV - RAV','Sofia Plus','Sofia Row','Sofia NA',
                 'Guam+ Row','Cebu','Minsk NA','Borneo NA','Odessa']

FamilyTrial_df = pd.DataFrame()
FamilyTrial_R_data_df = pd.DataFrame()  #------------------------- NEW

dates = []
dates = PlansDateInfo['DateInit'].tolist()

familyProducts = "'" + "', '".join(Moto_G_Family) + "'" 

for j in dates:
  #bq.updateFamilyQuerie((PlansDateInfo['PlanID'].iloc[x]),(PlansDateInfo['DateInit'].iloc[x]))
  bq.updateFamilyQuerie_01((PlansDateInfo['PlanID'].iloc[x]),(PlansDateInfo['DateInit'].iloc[x]),familyProducts)
  mt.preAPFD(bq.getPlanDataFamily())
  print("Numero de resultados analisados:")  #------------------------- NEW
  print(mt.history_size)  #------------------------- NEW
  df = APFDResults3(x,(PlansDateInfo['PlanID'].iloc[x]),(PlansDateInfo['DateInit'].iloc[x]))
  x+=1
  FamilyTrial_df = FamilyTrial_df.append(df)
  FamilyTrial_R_data_df = FamilyTrial_R_data_df.append(df2)  #------------------------- NEW


In [ ]:
Moto_G_Family = ['Rio H - Lake','Oslo M','Rio M - River','Oslo H - Ocean','Doha - Doha','RAV - RAV','Sofia Plus','Sofia Row','Sofia NA',
                 'Guam+ Row','Cebu','Minsk NA','Borneo NA','Odessa']

FamilyTrial_df = pd.DataFrame()
FamilyTrial_R_data_df = pd.DataFrame()  #------------------------- NEW

dates = []
dates = PlansDateInfo['DateInit'].tolist()

familyProducts = "'" + "', '".join(Moto_G_Family) + "'" 

for j in dates:
  bq.updateFamilyQuerie((PlansDateInfo['PlanID'].iloc[x]),(PlansDateInfo['DateInit'].iloc[x]))
  mt.preAPFD(bq.getPlanDataFamily())
  print("Numero de resultados analisados:")  #------------------------- NEW
  print(mt.history_size)  #------------------------- NEW

  print("debug")
  print(x)
  print(PlansDateInfo['PlanID'].iloc[x])
  print(PlansDateInfo['DateInit'].iloc[x])
  
  df = APFDResults3(x,(PlansDateInfo['PlanID'].iloc[x]),(PlansDateInfo['DateInit'].iloc[x]))
  
  x+=1
  FamilyTrial_df = FamilyTrial_df.append(df)
  FamilyTrial_R_data_df = FamilyTrial_R_data_df.append(df2)  #------------------------- NEW


In [ ]:
#mt.apfd_filtered.head()

In [ ]:
FamilyTrial_R_data_df.head(30)

In [ ]:
#FamilyTrial_df.loc['mean'] = FamilyTrial_df.mean()
FamilyTrial_df.head(30)
#len(FamilyTrial_df)

###Save output to HBPrio Trial Spreadsheet 

[HBPrio Trial Spreadsheet](https://docs.google.com/spreadsheets/d/11lHXTsAPC_40kiSXIjyHPl7RHRT6N2NzOjA41HdlsYM/edit#gid=1426537970)

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

#copia todo o workbook pra essa variavel 
wbfull = gc.open_by_key("11lHXTsAPC_40kiSXIjyHPl7RHRT6N2NzOjA41HdlsYM")

today = datetime.now().strftime('%d/%m/%Y|%H:%M')
newSheetName = 'Execution Record - '+ today+'(utc)'

wbfull.duplicate_sheet(1426537970,new_sheet_name=newSheetName)

#PASSANDO OS DADOS PRA NOVA ABA
newSheet = wbfull.worksheet(newSheetName)
plansInfosList = FamilyTrial_df.values.tolist()
newSheet.update('C3:S56',plansInfosList)

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

#copia todo o workbook pra essa variavel 
wbfull = gc.open_by_key("11lHXTsAPC_40kiSXIjyHPl7RHRT6N2NzOjA41HdlsYM")

today = datetime.now().strftime('%d/%m/%Y|%H:%M')
newSheetName = 'R Data - '+ today+'(utc)'

wbfull.duplicate_sheet(1563854476,new_sheet_name=newSheetName)

#PASSANDO OS DADOS PRA NOVA ABA
newSheet = wbfull.worksheet(newSheetName)
plansInfosList = FamilyTrial_R_data_df.values.tolist()
newSheet.update('B3:F',plansInfosList)

###Plota hotmap do exemplo acima

In [ ]:
import matplotlib.pyplot as plt
from pandas import DataFrame
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(12,10))
sns.heatmap(FamilyTrial_df[['OTD', 'ODD', 'RO', 'BO', 'OtN', 'NtO', 'OHB']], cmap="Blues", annot=True, linecolor='white', linewidths=0.5)